In [4]:
from leo_utils import arc_point_on_earth, compute_satellite_intersection_point_enu, compute_az_el_dist
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from minmax_solvers import (
    solve_game_mirror_prox,
    solve_game_bestresp_Q0_then_Q1,
    solve_game_proxBR,
    solve_game_extragradient,
    solve_game_pdhg,
    solve_game_proxBR_pp
)
# Your utils
from leo_utils import arc_point_on_earth, compute_satellite_intersection_point_enu
from sionna.rt import PlanarArray


import numpy as np
from numpy.linalg import cholesky, solve, eigh


c = 3e8  # speed of light (m/s)
fc = 10e9  # Carrier frequency: 10 GHz
wavelength = c / fc
bandwidth = 100e6  # 100 MHz
tx_power_dbm = 30  #  dBm
jam_power_dbm = 50
k = 1.38e-23  # Boltzmann 
GT = 13 # db gain-to-noise-temperature for for 0.33m Equivalent satellite antenna aperturesatellites, or can be 5 dB K^(-1) for 0.13m Equivalent satellite antenna aperture
La = 5 # dB
GT_linear_inv = 10 ** (-GT / 10)
La_linear = 10 ** (La / 10)
noise_power_watt = k * bandwidth * GT_linear_inv * La_linear
Tx_power_watt = 10 ** ((tx_power_dbm  - 30)/ 10)  
Jam_power_watt = 10 ** ((jam_power_dbm  - 30)/ 10)

jitc_llvm_init(): LLVM API initialization failed ..


In [5]:
from sionna.rt import Scene, Receiver, Transmitter, PlanarArray, PathSolver
import numpy as np
import vsat_dish_3gpp


def compute_cir(tx_pos, rx_pos, tx_array, rx_array,tx_look_at, rx_look_at, frequency=10e9):
    """
    Compute CIR from a single transmitter to multiple receivers using synthetic arrays.
    
    Args:
        tx_pos:        (3,) list or np.array for transmitter position
        rx_pos_array:  (m,3) array of receiver positions
        tx_array:      PlanarArray for the transmitter
        rx_array:      PlanarArray for all receivers (shared)
        frequency:     Frequency in Hz

    Returns:
        a_list:    list of CIR amplitude arrays, one per RX
        tau_list:  list of delay arrays, one per RX
    """
    scene = Scene()
    scene.frequency = frequency
    scene.synthetic_array = True

    # Add transmitter
    scene.tx_array = tx_array
    tx = Transmitter(name="tx", position=tx_pos, display_radius=200)
    scene.add(tx)

    # Add receivers
    scene.rx_array = rx_array
    rx_list = []
    for i, rx_pos in enumerate(rx_pos):
        rx = Receiver(name=f"rx{i}", position=rx_pos)
        scene.add(rx)
        rx.look_at(rx_look_at)
        rx_list.append(rx)

    tx.look_at(tx_look_at)  # Point TX to the first RX

    # Solve paths
    solver = PathSolver()
    paths = solver(scene=scene,
                   max_depth=0,
                   los=True,
                   synthetic_array=True,
                   seed=41)

    # Get CIRs
    a_all, tau_all = paths.cir(normalize_delays=False, out_type="numpy")

    for tx_name in scene.transmitters:
        scene.remove(tx_name)
    for rx_name in scene.receivers:
        scene.remove(rx_name) 
    
    return a_all, tau_all

jam_rows =8
jam_cols = 8
jam_antennas = jam_cols*jam_rows

sat_rows = 1
sat_cols = 1
sat_antennas = sat_cols*sat_rows

tx_rows = 8
tx_cols = 8
tx_antennas = tx_cols*tx_rows

tx_array = PlanarArray(num_rows=tx_rows, num_cols=tx_cols,
                        vertical_spacing=0.5, horizontal_spacing=0.5,
                        pattern="tr38901", polarization="V")
                        # pattern="iso", polarization="V")

jam_array = PlanarArray(num_rows=jam_rows, num_cols=jam_cols,  
                            vertical_spacing=0.5, horizontal_spacing=0.5,
                        #  pattern="vsat_dish",
                            pattern="tr38901",
                            polarization="V")

sat_array = PlanarArray(num_rows=sat_rows, num_cols=sat_cols,
                             vertical_spacing=0.5, horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

def collapse_channel(a_cir, t_idx=0):
    """
    Collapse channel tensor at a given time index.
    Input:
      a_cir: [num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]
    Output:
      H_t: (M, N), M = num_rx*num_rx_ant, N = num_tx*num_tx_ant
    """
    nr, nra, nt, nta, npaths, ntimes = a_cir.shape
    assert 0 <= t_idx < ntimes
    H_t = a_cir[..., t_idx]             # select time slice
    H_t = H_t.sum(axis=-1)              # sum over paths
    H_t = H_t.reshape(nr*nra, nt*nta)   # merge Rx/Tx antennas
    return H_t


In [1]:
import pandas as pd
# Load and make sure each time block is ordered by Rank (nearest first)
df = pd.read_csv("tracked_satellites_tol0.csv")
df = df.sort_values(["Time", "Rank"], ascending=[True, True]).reset_index(drop=True)

In [2]:

n_steps = df["Time"].nunique()
print("Total steps:", n_steps)
max_steps = 3
print(df.head(max_steps*5))

Total steps: 200
                   Time                  Name  Action  Rank  Slant km  \
0   2025-10-06 21:14:34  STARLINK-11640 [DTC]    seed     1     424.8   
1   2025-10-06 21:14:34        STARLINK-32375    seed     2     496.8   
2   2025-10-06 21:14:34        STARLINK-34404    seed     3     508.9   
3   2025-10-06 21:14:34        STARLINK-31872    seed     4     528.4   
4   2025-10-06 21:14:34        STARLINK-33752    seed     5     556.4   
5   2025-10-06 21:35:05        STARLINK-32438  reseed     1     499.2   
6   2025-10-06 21:35:05        STARLINK-32699  reseed     2     500.9   
7   2025-10-06 21:35:05        STARLINK-31657  reseed     3     533.1   
8   2025-10-06 21:35:05        STARLINK-32961  reseed     4     568.4   
9   2025-10-06 21:35:05        STARLINK-30321  reseed     5     570.2   
10  2025-10-06 21:36:40  STARLINK-11647 [DTC]  reseed     1     380.2   
11  2025-10-06 21:36:40         STARLINK-4524  reseed     2     549.5   
12  2025-10-06 21:36:40        STA

In [6]:
# Generate RX positions

distances_km = [2]
azimuths_deg = np.linspace(0, 360, len(distances_km), endpoint=False)
gnd_positions = [np.array([0.0, 0.0, 0.0])]

for d_km, az in zip(distances_km, azimuths_deg):
    pos = arc_point_on_earth(d_km, az)
    gnd_positions.append(pos)
gnd_positions = np.array(gnd_positions)

for i, pos in enumerate(gnd_positions):
    print(f"TX{i}(m): {pos}")

TX0(m): [0. 0. 0.]
TX1(m): [ 1.22464678e-13  1.99999997e+03 -3.13922458e-01]


In [ ]:
from tqdm import tqdm

per_time = {}
frames = []
max_steps = n_steps
N0 = noise_power_watt
P0 = Tx_power_watt
P1 = Jam_power_watt

# methods = ["BestResp", "MirrorProx", "ProxBR", "ProxBRpp"]
methods = [ "MirrorProx"]
sat_counts = [1,  3, 5] 
results = {m: {k: [] for k in sat_counts} for m in methods}
groups = list(df.sort_values(["Time","Rank"]).groupby("Time"))


for step_idx, (t, g) in enumerate(tqdm(groups, desc="Processing steps"), start=1):
    # If more/less than k are present, we still proceed with available rows
    gk = g.sort_values("Rank").head(k).copy()

    # Build sat_positions in ENU (meters)
    sat_positions = gk[["x_East (m)", "y_North (m)", "z_Up (m)"]].to_numpy()  # (K,3)
    names = gk["Name"].tolist()
    K_all = sat_positions.shape[0]
    

    a_tx,  tau_tx  = compute_cir(gnd_positions[0], sat_positions, tx_array, sat_array, sat_positions[i], gnd_positions[0])
    a_jam, tau_jam = compute_cir(gnd_positions[1], sat_positions, tx_array, sat_array, sat_positions[i], gnd_positions[0])
    
    H0_full = collapse_channel(a_tx)
    H1_full = collapse_channel(a_jam)

    
    for k_sel in sat_counts:
        if K_all < k_sel:
            continue  # 该时刻不足 k_sel 颗，跳过
        
        
        # 取前 k_sel 颗卫星对应的行块（按 Rank 的顺序）
        H0 = H0_full[:k_sel*sat_antennas, :]
        H1 = H1_full[:k_sel*sat_antennas, :]

        # 均分初始化 Q1
        n1 = H1.shape[1]
        Q1_init = (P1/n1) * np.eye(n1, dtype=complex)
        
    for m in methods:
        if m == "ProxBRpp":
            Q0, Q1, hist = solve_game_proxBR_pp(
                H0, H1, N0, P0, P1,
                rho=1e-2,
                outer_steps=200, outer_tol=1e-6,
                inner_max=500, inner_tol=1e-6,
                eta0_inner=0.5,
                beta=0.3, gamma=1.05,
                eta_min=1e-2, eta_max=0.3,
                min_outer=5,
                verbose=False, track_hist=True,
                # 调参：
                sigma=0.1, tau=5e-3, grow=1.05,
                lam=1.0, pg_eta=0.1, proj_mode='eq',
                rho_max=5e-2, freeze_backtrack_at=5e-5,
                Q1_init=Q1_init
            )
        elif m == "ProxBR":
            Q0, Q1, hist = solve_game_proxBR(
                H0, H1, N0, P0, P1,
                rho=1e-2,
                outer_steps=400, outer_tol=1e-6,
                inner_max=300, inner_tol=1e-6,
                eta0_inner=0.5,
                beta=0.1, gamma=1.05,
                eta_min=1e-1, eta_max=1.0,
                eta_probe=0.1,
                min_outer=5,
                verbose=False, track_hist=True
            )
        elif m == "MirrorProx":
            Q0, Q1, hist = solve_game_mirror_prox(
                H0, H1, N0, P0, P1,
                steps=8000,
                eta=0.25, step_rule='adp',
                beta=0.3, gamma=1.03,
                eta_min=1e-3, eta_max=1.0,
                strong_reg=0.0,
                Q0_init=None, Q1_init=Q1_init,
                use_averaging=True, eta_probe=0.2,
                verbose=False, track_hist=True
            )
        elif m == "BestResp":
            Q0, Q1, n_it, hist = solve_game_bestresp_Q0_then_Q1(
                H0, H1, N0,
                P0=P0, P1=P1,
                max_outer=1000, tol=1e-6, inner_Q1_steps=4,
                geometry='entropy', step_rule='adp',
                eta=0.5, eta_init=0.3,
                eta_min=1.5e-1, eta_max=0.9,
                beta=0.1, gamma=1.01,
                multi_stream=True,
                verbose=False, track_hist=True,
                Q1_init=Q1_init
            )
        else:
            raise ValueError(f"Unknown method {m}")
        
        results[m][k_sel].append({
                "time": pd.Timestamp(t).isoformat() if not pd.isna(t) else None,
                "iters": len(hist.get("J", [])) if isinstance(hist, dict) else None,
                "hist"
            })
    
    if step_idx >= max_steps:
        break



NameError: name 'noise_power_watt' is not defined